**Implementación del problema**: Rompecabezas lineal de 4 cifras.

**Problema**: Dada una lista de 4 dígitos, encuentra la secuencia de acciones 
mínima para reordenar la lista de manera que se cumpla una condición dada.

**Ejemplo**: Dada la lista [3,1,2,4], encuentra la secuencia mínima de acciones
para ordenarla en sentido ascendente [1,2,3,4]

**Acciones permitidas**:
ii -> Intercambia el dígito 1 por el dígito 2
ic -> Intercambia el dígito 2 por el dígito 3
id -> Intercambia el dígito 3 por el dígito 4

**Solución**:

¡Solución encontrada!
Estado inicial: [3,1,2,4]

Información de nodo solución:
ID: J8r9Q/MSiMsYq3lH6zy1kgUG
Estado: [1,2,3,4]
ID-Padre: aOf*Q0+3x/pIG5IdrPPDTmpq
Op Generador: ic
Acciones solucion: ic,ii

Tiempo de ejecución: 0.0011432170867919922


In [131]:
# Dependencias
import random
import string
import time

In [119]:
# Implementación del Estado para problema de recompecabezas lineal de 4 cifras
# así como los operadores que se pueden utilizar para cambiar de un estado
# a otro. 
# Estado se implementa como clase. 
# Operador se implementa como propiedad de la clase.
# Estructura de datos: estado -> [a,b,c,d]
# Estructura de datos: operador -> [etiqueta, acción(fn)]
class Estado:
  def __init__(self,a,b,c,d):
    self.__a = a
    self.__b = b
    self.__c = c
    self.__d = d

    __op1 = ['ii',self.mov_ii]
    __op2 = ['ic',self.mov_ic]
    __op3 = ['id',self.mov_id]

    self.__operadores = [__op1,__op2,__op3]

# Getter de estado en formato lista
  def get_sts(self):
    return [self.__a,self.__b,self.__c,self.__d]

# Getter de un dígito concreto de un estado
  def get_digit(self,index):
    if index < 0 or index > 3:
      return "[Error] Index number is lower than 0 or greater than 3"
    return self.get_sts()[index]

# Getter de operadores
  def get_op(self):
    return self.__operadores

# Acciones posibles (3) para este problema. 

# mov_ii (izquierda) intercambia el dígito 1 y el dígito 2
  def mov_ii(self):
    self.__copiar_estado(Estado(self.__b,self.__a,self.__c,self.__d))

# mov_ic (centro) intercambia el dígito 2 y el dígito 3
  def mov_ic(self):
    self.__copiar_estado(Estado(self.__a,self.__c,self.__b,self.__d))

# mov_id (derecha) intercambia el dígito 3 y el dígito 4
  def mov_id(self):
    self.__copiar_estado(Estado(self.__a,self.__b,self.__d,self.__c))

# devuelve un estado incorrecto [0,0,0,0] en caso de error
  def delete_sts(self):
    self.__copiar_estado(Estado(0,0,0,0))

# Función auxiliar para las acciones de cambio de estado
  def __copiar_estado(self,estado):
    self.__a = estado.get_digit(0)
    self.__b = estado.get_digit(1)
    self.__c = estado.get_digit(2)
    self.__d = estado.get_digit(3)

# Función que realiza una acción en función del operador pasado como parámetro
# Cambia el estado a [0,0,0,0] si el tag introducido no corresponde a 
# ninguna acción existente.
  def __mov(self,op_tag):
    for op in self.__operadores:
      if op[0] == op_tag:
        return getattr(self, "mov_" + str(op[0]), lambda: "Invalid tag")()
    return self.delete_sts()

# Función que devuelve una instancia de estado tras aplicar una acción
  def doAction(self,op_tag):
    aux_sts = Estado(self.get_digit(0),
                     self.get_digit(1),
                     self.get_digit(2),
                     self.get_digit(3))
    aux_sts.__mov(op_tag)
    return aux_sts

# Función que compara dos estados y devuelve boolean
  def isEqual(self,estado):
    return self.get_sts() == estado.get_sts()

  def toString(self):
    return str("[" + str(self.get_digit(0)) + "," +
               str(self.get_digit(1)) + "," +
               str(self.get_digit(2)) + "," +
               str(self.get_digit(3)) + "]")


In [3]:
# Implementación de "Problema" donde se guardan las particularidades del problema
class Problema:
  def __init__(self, estado_inicial, funcion_objetivo, info_adicional = []):
    self.__estado_inicial = estado_inicial
    self.__funcion_objetivo = funcion_objetivo
    self.__info_adicional = info_adicional

# Getter de estado inicial
  def get_sts_ini(self):
    return self.__estado_inicial

# Getter de función objetivo
  def get_fn_obj(self):
    return self.__funcion_objetivo

# Getter de información adicional
  def get_inf_add(self):
    return self.__info_adicional

# Dado un nodo pasado como parámetro, devuelve True
# si el nodo es solución al problema (satisface la función
# objetivo) o False si no lo hace
  def esSolucion(self,nodo):
    return self.get_fn_obj()(nodo.get_sts())

In [74]:
# Implementación estructura de datos nodo

class Nodo:
  def __init__(self, 
               estado, 
               id_padre='none', 
               operador_generador='none', 
               informacion_adicional = []):
    self.__id = Nodo.symgen(24)
    self.__sts = estado
    self.__id_padre = id_padre
    self.__op_gen = operador_generador
    self.__inf_add = informacion_adicional

# Getter del id del nodo
  def get_id(self):
    return self.__id

# Getter de estado del nodo
  def get_sts(self):
    return self.__sts

# Getter del id_padre
  def get_id_padre(self):
    return self.__id_padre

# Getter de la operación generadora
  def get_op_gen(self):
    return self.__op_gen

# Getter de la información adicional
  def get_inf_add(self):
    return self.__inf_add

# Genera una lista de la expansión del nodo. 
# Los nuevos nodos generados son el resultado de aplicar cada operacion
# válidas al estado del nodo a expandir.
# De esta forma, se generará una lista de nodos de longitud igual
# al número de operaciones con resultado válido.
# En este método se podrán añadir restricciones a la expansión. Por ejemplo,
# podremos evitar que los nodos resultantes sean resultado de aplicar una
# acción que sea igual a la aplicada para generar el nodo padre o nodo de 
# expansión.
  def expandir(self):
    # Restricción que impide aplicar una acción ya aplicada en la generación
    # del nodo padre. El índice [0] hace referencia a la etiqueta del operador
    # Recordamos que la estructura de datos operador está compuesta por 
    # una etiqueta [tag] y una función() [accion()].
    nuevas_hojas = []

    # op_gen_tag es la etiqueta del operador generador del nodo que vamos a 
    # expandir.
    op_gen_tag = self.get_op_gen()[0]

    # Iteración que generará un nodo por cada operador existente
    for op in self.get_sts().get_op():

      # Restricción que impide que se genere un nodo a partir de la misma 
      # acción que se utilizó para generar al nodo padre.
      if op[0] != op_gen_tag:
        # Genera un nuevo nodo a partir de aplicar la acción al estado,
        # el id del nodo padre (o el nodo que estamos expandiendo),
        # y el operador utilizado.
        nuevas_hojas.append(Nodo(self.get_sts().doAction(op[0]),
                                 self.get_id(),
                                 op[0]))
    return nuevas_hojas    
      
# Genera una cadena de alfanuméricos + simbolos aleatorio de longitud igual 
# al valor pasado por parámetro.
  def symgen(n_digits):
    symbol = ''
    for i in range(0,n_digits):
      symbol = symbol + (random.choice(string.ascii_letters + string.digits + str('*/|+$%')))
    return symbol

# Imprime la información de un nodo dado
  def show_brief(self):
    print ("ID: " + self.get_id())
    print ("Estado: " + self.get_sts().toString())
    print ("ID-Padre: " + self.get_id_padre())
    print ("Op Generador: " + self.get_op_gen())
    ele_numb = 0
    for element in self.get_inf_add():
      ele_numb += 1
      print ("Info adicional [" + ele_numb + "]: " + str(element))



In [112]:
# Implementación estructura de datos árbol de búsqueda
# nodos es la lista de nodos ya expandidos (y procesados)
# que forman parte del árbol de búsqueda.
# hojas es la cola ordenada o con prioridad de nodos aún no expandidos
# ** NOTA **
# Es importante tener en cuenta que la estrategia de búsqueda determinará
# el orden con el cual se incluyen las hojas, y esto el comportamiento de
# la búsqueda (y eficiencia del algoritmo).
# ** NOTA **
class Arbol:
  def __init__(self, nodo_inicial):
    self.nodos = []
    self.hojas = [nodo_inicial]

# Esta función añade una lista de nodos a la cola ordenada de hojas
# teniendo en cuenta la estrategia de busqueda
  def añadirHojas(self,nuevas_Hojas,indexOrTag_EstrategiaBusqueda):
    # Sobreescribe la cola de hojas con la generada por la función 
    # estrategia_de_busqueda. Esta función añadirá las nuevas_Hojas
    # a la cola con prioridad teniendo en cuenta distintos factores,
    # dependiendo de la estrategia utilizada.
    self.hojas = estrategia_de_busqueda(indexOrTag_EstrategiaBusqueda, 
                                        self.hojas, 
                                        nuevas_Hojas)

# Esta función devuelve true si hay hojas pendientes de procesar,
# o false en caso de no haberlas.
  def hayHojas(self):
    return len(self.hojas[:]) > 0

# Devuelve la secuencia de acciones desde el nodo raiz hasta el
# nodo solución.
  def camino(self,nodo_solucion):
    nodo_sel = nodo_solucion
    camino = "Acciones solucion: " + nodo_sel.get_op_gen()
    raizEncontrada = False
    # Mientras no se haya encontrado el nodo raiz
    while raizEncontrada == False:
      # Busca el nodo padre en la lista de nodos procesados
      for nodo in reversed(self.nodos):
        # Cuando se encuentre el nodo padre
        if nodo.get_id() == nodo_sel.get_id_padre():
          # Selecciona al nodo padre
          nodo_sel = nodo

          # Si el nodo seleccionado no es raiz:
          if nodo_sel.get_op_gen() != 'none':
            # Añade el movimiento generador al camino
            camino = camino + "," + (nodo_sel.get_op_gen())
            break
          # Si el nodo seleccionado es raiz, termina
          else:
            raizEncontrada = True
          
    return camino


In [148]:
# Algoritmo de búsqueda

def busqueda(problema, indexOrTag_EstrategiaBusqueda, arbol):
  # Variable auxiliar para medir el tiempo de ejecución
  start = time.time()

  # Variable auxiliar para limitar el número de iteraciones máximas
  limite = 0
  limite_max = 15000

  # Variable auxiliar que detendrá el bucle una vez se haya encontrado
  # la solución
  solucion_encontrada = False

  # Declaración de variables arbol y nodo para guardar las modificaciones
  # que se realicen en el ámbito del bucle.
  arbol = arbol
  nodo = []

  # Bucle principal del algoritmo
  while not solucion_encontrada and arbol.hayHojas() and limite < limite_max:
    limite += 1
    # Selecciona la primera hoja sin procesar y la elimina de
    # la cola de hojas no procesadas
    nodo = arbol.hojas.pop(0)

    # Añade la hoja a la lista de nodos procesados
    arbol.nodos.append(nodo)

    # Comprueba si el nodo seleccionado es solución
    if problema.esSolucion(nodo):
      solucion_encontrada = True

    # Si el nodo seleccionado no es solución, lo expande generando sus hijos,
    # y los añade a la cola de hojas sin procesar
    else:
      # Genera una lista con las nuevas hojas generadas a partir del nodo
      # seleccionado.
      nuevos_nodos = nodo.expandir()

      # Añade las nuevas hojas a la lista de hojas sin procesar teniendo
      # en cuenta la estrategia de busqueda, la cual determinará el orden
      # en el que las hojas serán añadidas.
      arbol.añadirHojas(nuevos_nodos,
                        indexOrTag_EstrategiaBusqueda)

  if solucion_encontrada:
    print("¡Solución encontrada!")
    print("Estado inicial: " + problema.get_sts_ini().toString())
    print("\nInformación de nodo solución:")
    nodo.show_brief()
    print(arbol.camino(nodo))
    print("\nTiempo de ejecución: " + str(time.time()-start))
    return arbol
  else:
    print("Solución no encontrada")
    return []

In [25]:
# Estrategias de búsqueda
# Entrada: indexOrTag [Index or String] 
#          nuevas_hojas [lista de nodos] y arbol.hojas [lista de nodos]

# Esta función aplica una determinada estrategia de búsqueda
# Funciona como un switch/select case
# Acepta un entero o un string con el tag de la estrategia
def estrategia_de_busqueda(indexOrTag, hojas, nuevas_hojas):
  strategy = {1: BFS(hojas,nuevas_hojas),
              }

  if isinstance(indexOrTag,int):
    return strategy.get(indexOrTag, lambda: "[ERROR] Index out of bounds")

  elif isinstance(indexOrTag,str):
    if indexOrTag == 'BFS':
      return strategy.get(1, lambda: "[ERROR] Tag not found")

  else:
    return '[ERROR] index or tag not recogniced'

# BFS - Breadth First Search
def BFS(hojas,nuevas_hojas):
  aux = hojas
  for hoja in nuevas_hojas:
    aux.append(hoja)
  return aux


In [150]:
# Implementación de un problema concreto

# *******************
# ** Configuración **

# Establece la función objetivo, que verificará la solución encontrada.
# Entrada: estado a comprobar
funcion_objetivo = lambda sts1: sts1.isEqual(Estado(1,2,3,4))

# Estado inicial del que se parte
estado_inicial = Estado(3,1,2,4)

# Indice de estrategia de solución
indexOrTag = 'BFS'

# ** Fin configuración **
# ***********************

arbol_solucion = busqueda(Problema(estado_inicial, funcion_objetivo),indexOrTag,Arbol(Nodo(estado_inicial)))

¡Solución encontrada!
Estado inicial: [3,1,2,4]

Información de nodo solución:
ID: J8r9Q/MSiMsYq3lH6zy1kgUG
Estado: [1,2,3,4]
ID-Padre: aOf*Q0+3x/pIG5IdrPPDTmpq
Op Generador: ic
Acciones solucion: ic,ii

Tiempo de ejecución: 0.0011432170867919922
